# Scrapy
- 웹사이트에서 데이터 수집을 위한 오픈소스 파이썬 프레임워크
- 멀티스레딩으로 데이터 수집
- gmarket 상품데이터 수집

In [1]:
# install scrapy
#!pip install scrapy

## 1. make project

In [2]:
!scrapy startproject news

Error: scrapy.cfg already exists in C:\Users\tjdud\web\news


In [3]:
!tree news

폴더 PATH의 목록입니다.
볼륨 일련 번호가 000000E7 22D1:176C입니다.
C:\USERS\TJDUD\WEB\NEWS
└─news
    └─spiders


- scrapy structure
    - items : 데이터의 모양 정의
    - middewares : 수집할때 header 정보와 같은 내용을 설정
    - pipelines : 데이터를 수집한 후에 코드를 실행
    - settings : robots.txt 규칙, 크롤링 시간 텀등을 설정
    - spiders : 크롤링 절차를 정의

## 2. xpath
- link, contents

In [4]:
import scrapy, requests
from scrapy.http import TextResponse

In [5]:
# 링크 데이터

In [6]:
request = requests.get('https://news.daum.net')
response = TextResponse(request.url, body=request.text, encoding='utf-8')
response

<200 https://news.daum.net/>

In [7]:
selector = '/html/body/div[2]/main/section/div/div[1]/div[1]/ul/li'
selector += '/div/div/strong/a/@href'
links = response.xpath(selector).extract()
len(links), links[:2]

(20,
 ['https://v.daum.net/v/20240923150234822',
  'https://v.daum.net/v/20240923143506311'])

In [8]:
# 상세 데이터

In [9]:
link = links[19]
request = requests.get(link)
response = TextResponse(request.url, body=request.text, encoding='utf-8')
response

<200 https://v.daum.net/v/20240923151602477>

In [17]:
title = response.xpath('//*[@id="mArticle"]/div[1]/h3/text()')[0].extract()
content = response.xpath('//section//p/text()').extract()
content = ' '.join(content).replace('\xa0', ' ').replace("\'", ' ')
title, content[:100]

('직장인들 국민은행 이용 편해진다',
 'KB국민은행이 23일부터  점심시간 집중상담  서비스 운영 지점을 늘렸다. 점심시간 집중상담 지점은 KB국민은행 홈페이지 내  지점 안내  또는 모바일뱅킹 앱 KB스타뱅킹 내  점')

## 3. items.py
- Data Model

In [18]:
%%writefile news/news/items.py
import scrapy

class NewsContents(scrapy.Item):
    title = scrapy.Field()
    content = scrapy.Field()
    link = scrapy.Field()

Overwriting news/news/items.py


## 4. spider.py
- wirte crawling process

In [15]:
%%writefile news/news/spiders/spider.py
import scrapy
from news.items import NewsContents


class NewsSpider(scrapy.Spider):
    name = 'news'
    allow_domain = ['daum.net']
    start_urls = ['https://news.daum.net']
    
    def parse(self, response):
        selector = '/html/body/div[2]/main/section/div/div[1]/div[1]/ul/li'
        selector += '/div/div/strong/a/@href'
        links = response.xpath(selector).extract()
        for link in links:
            yield scrapy.Request(link, callback=self.parse_content)
            
    def parse_content(self, response):
        item = NewsContents()
        item['title'] = response.xpath(
            '//*[@id="mArticle"]/div[1]/h3/text()')[0].extract()
        item['link'] = response.url
        content = response.xpath('//section//p/text()').extract()
        content = ' '.join(content).replace('\xa0', ' ').replace("\'", ' ')
        item['content'] = content
        yield item

Overwriting news/news/spiders/spider.py


## 5. run scrapy
- news 디렉토리에서 아래의 커멘드 실행
- scrapy crawl news -o news.csv

In [16]:
import pandas as pd
pd.read_csv("news/news.csv")[['title', 'link', 'content']].tail(2)

FileNotFoundError: [Errno 2] No such file or directory: 'news/news.csv'